# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

# Imports

In [ ]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display, Audio
from openai import OpenAI
import anthropic
import base64
from io import BytesIO
from PIL import Image
import gradio as gr

# OpenAI API key

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")



In [ ]:
openai = OpenAI()

claude = anthropic.Anthropic()

# SYSTEM prompt

In [ ]:
# set up environment

def system_prompt(selected_model):
    return f"""
    You are a tech expert and know every coding language, and can give 
    nice, detailed and simple explanations for the given questions.
    Introduce yourself by saying which model you are every time you answer. For example, this is {selected_model}.  
    
    """

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    display(Audio(output_filename, autoplay=True))

In [ ]:
def listener(audio_file):
    with open(audio_file, "rb") as audio:
        transcript = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio
        )
    return transcript.text

In [ ]:
def chat(cleared_entry, history, selected_model):
    messages = [{"role": "system", "content": system_prompt(selected_model)}] + history

    print(messages)
    
    if selected_model == "GPT-4o-mini":
        stream = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, stream=True)
        response = ""
        
        for chunk in stream:
            try:
                response += chunk.choices[0].delta.content or ''
                
                updated_history = history + [{"role": "assistant", "content": response}]
                # talker(response)
                yield updated_history, None  
            except Exception as e:
                print(f"Streaming error: {e}")
                yield "Sorry, there was an error processing your request."
        # talker(response)
    elif selected_model == "Claude-sonnet-4":
        claude_messages = [{"role": msg["role"], "content": msg["content"]} for msg in history]
        print(claude_messages)
        result = claude.messages.stream(
            model="claude-sonnet-4-20250514",
            max_tokens=200,
            temperature=0.7,
            system=system_prompt(selected_model),  
            messages=claude_messages,  
        )
        
        response = ""
        with result as stream:
            for text in stream.text_stream:
                response += text
              
                updated_history = history + [{"role": "assistant", "content": response}]
                
                yield updated_history, None  
        # talker(response)

In [ ]:
with gr.Blocks() as ui:

    gr.Markdown("## AI Chat Assistant")
    gr.Markdown("**Select your preferred AI model:**")
    
    model_dropdown = gr.Dropdown(
        choices=["GPT-4o-mini", "Claude-sonnet-4"], 
        value="GPT-4o-mini",  # default selection
        label="Choose Model"
    )

    
    with gr.Row():
        chatbot = gr.Chatbot(height=200, type="messages")
        image_output = gr.Image(height=200)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        # Audio input for voice messages
        audio_input = gr.Audio(
            sources=["microphone", "upload"], 
            type="filepath", 
            label="🎙️ Voice Message"
        )
    with gr.Row():
        voice_submit = gr.Button("Send Voice Message", variant="secondary")
        clear = gr.Button("Clear")


    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    def process_voice_input(audio_file):
        """Convert voice to text and put it in the text box"""
        if audio_file is not None:
            transcribed_text = listener(audio_file)
            if transcribed_text and not transcribed_text.startswith("Error"):
                return transcribed_text
        return ""

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[entry,chatbot, model_dropdown], outputs=[chatbot, image_output]
    )

    voice_submit.click(
        process_voice_input,
        inputs=[audio_input],
        outputs=[entry]
    )
    
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)